In [33]:
!pip install wget
!pip install spacy
!pip install gensim
!pip install pyspellchecker
!python -m spacy download en_core_web_trf

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [34]:
# !wget -O data.csv https://www.dropbox.com/s/xd35vnpmi2f5gdb/2016_US_election_tweets_100k.csv?dl=0
!wget -O data.zip "https://www.dropbox.com/s/szuhoq38fk82u5n/hw2_data.zip?dl=0"
!rm -rf Data_file
!unzip data.zip

--2021-04-09 12:34:54--  https://www.dropbox.com/s/szuhoq38fk82u5n/hw2_data.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:601c:18::a27d:612, 162.125.4.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:601c:18::a27d:612|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/szuhoq38fk82u5n/hw2_data.zip [following]
--2021-04-09 12:34:54--  https://www.dropbox.com/s/raw/szuhoq38fk82u5n/hw2_data.zip
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc42d6783e70f1ce2bf2ac9c450e.dl.dropboxusercontent.com/cd/0/inline/BMSbbUaOP0y25leluWvZyUid0ETIywYmcCqs9APHeqHL__X5LKs_b47bawPheOo6crSL0xXfXnN_sVW_9_AaU-VHvdolammuuhXL_lGf_Hq0Egdeq7zWRhV9n5OH5zFn68H-CYOLZNdH65dw61uZG_Il/file# [following]
--2021-04-09 12:34:55--  https://uc42d6783e70f1ce2bf2ac9c450e.dl.dropboxusercontent.com/cd/0/inline/BMSbbUaOP0y25leluWvZyUid0ETIywYmcCqs9APHeqHL__X5LKs_b47bawPheOo6crSL0

In [35]:
import multiprocessing
import re
import string

from spellchecker import SpellChecker
import pandas as pd

import spacy

import nltk;
from nltk.corpus import stopwords
nltk.download('stopwords')

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dipta007/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
dfs = []
for i in range(1, 6):
  df = pd.read_csv(f'./Data_file/#longcovid{i}.csv', header=None, names=['data', 'tweet_text'])
  dfs.append(df)

df = pd.concat(dfs)

df.head()

In [ ]:
len(df)

In [ ]:
df = df.dropna(subset=['tweet_text'])

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
def clean_url(text):
    return re.sub(r"http\S+", "", text)

def clean(tweet): 
            
    # Special characters
    tweet = re.sub(r"\x89Û_", "", tweet)
    tweet = re.sub(r"\x89ÛÒ", "", tweet)
    tweet = re.sub(r"\x89ÛÓ", "", tweet)
    tweet = re.sub(r"\x89ÛÏWhen", "When", tweet)
    tweet = re.sub(r"\x89ÛÏ", "", tweet)
    tweet = re.sub(r"China\x89Ûªs", "China's", tweet)
    tweet = re.sub(r"let\x89Ûªs", "let's", tweet)
    tweet = re.sub(r"\x89Û÷", "", tweet)
    tweet = re.sub(r"\x89Ûª", "", tweet)
    tweet = re.sub(r"\x89Û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89Û¢", "", tweet)
    tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
    tweet = re.sub(r"fromåÊwounds", "from wounds", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"JapÌ_n", "Japan", tweet)    
    tweet = re.sub(r"Ì©", "e", tweet)
    tweet = re.sub(r"å¨", "", tweet)
    tweet = re.sub(r"SuruÌ¤", "Suruc", tweet)
    tweet = re.sub(r"åÇ", "", tweet)
    tweet = re.sub(r"å£3million", "3 million", tweet)
    tweet = re.sub(r"åÀ", "", tweet)
    
    # Contractions
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"don\x89Ûªt", "do not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I\x89Ûªm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "Here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you\x89Ûªve", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"it\x89Ûªs", "it is", tweet)
    tweet = re.sub(r"doesn\x89Ûªt", "does not", tweet)
    tweet = re.sub(r"It\x89Ûªs", "It is", tweet)
    tweet = re.sub(r"Here\x89Ûªs", "Here is", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"I\x89Ûªve", "I have", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"can\x89Ûªt", "cannot", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"wouldn\x89Ûªt", "would not", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"That\x89Ûªs", "That is", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"You\x89Ûªre", "You are", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"Don\x89Ûªt", "Do not", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"Can\x89Ûªt", "Cannot", tweet)
    tweet = re.sub(r"you\x89Ûªll", "you will", tweet)
    tweet = re.sub(r"I\x89Ûªd", "I would", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"i've", "I have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"ain't", "am not", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"I've", "I have", tweet)
    tweet = re.sub(r"Don't", "do not", tweet)
    tweet = re.sub(r"I'll", "I will", tweet)
    tweet = re.sub(r"I'd", "I would", tweet)
    tweet = re.sub(r"Let's", "Let us", tweet)
    tweet = re.sub(r"you'd", "You would", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"Ain't", "am not", tweet)
    tweet = re.sub(r"Haven't", "Have not", tweet)
    tweet = re.sub(r"Could've", "Could have", tweet)
    tweet = re.sub(r"youve", "you have", tweet)  
    tweet = re.sub(r"donå«t", "do not", tweet)   
            
    # Character entity references
    tweet = re.sub(r"&gt;", ">", tweet)
    tweet = re.sub(r"&lt;", "<", tweet)
    tweet = re.sub(r"&amp;", "&", tweet)
    
    # Typos, slang and informal abbreviations
    tweet = re.sub(r"w/e", "whatever", tweet)
    tweet = re.sub(r"w/", "with", tweet)
    tweet = re.sub(r"USAgov", "USA government", tweet)
    tweet = re.sub(r"recentlu", "recently", tweet)
    tweet = re.sub(r"Ph0tos", "Photos", tweet)
    tweet = re.sub(r"amirite", "am I right", tweet)
    tweet = re.sub(r"exp0sed", "exposed", tweet)
    tweet = re.sub(r"<3", "love", tweet)
    tweet = re.sub(r"amageddon", "armageddon", tweet)
    tweet = re.sub(r"Trfc", "Traffic", tweet)
    tweet = re.sub(r"8/5/2015", "2015-08-05", tweet)
    tweet = re.sub(r"WindStorm", "Wind Storm", tweet)
    tweet = re.sub(r"8/6/2015", "2015-08-06", tweet)
    tweet = re.sub(r"10:38PM", "10:38 PM", tweet)
    tweet = re.sub(r"10:30pm", "10:30 PM", tweet)
    tweet = re.sub(r"16yr", "16 year", tweet)
    tweet = re.sub(r"lmao", "laughing my ass off", tweet)   
    tweet = re.sub(r"TRAUMATISED", "traumatized", tweet)
    
    # Urls
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
        
    # Words with punctuations and special characters
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    for p in punctuations:
        tweet = tweet.replace(p, f' {p} ')
        
    # ... and ..
    tweet = tweet.replace('...', ' ... ')
    if '...' not in tweet:
        tweet = tweet.replace('..', ' ... ')
        
    return tweet

def clean_html(tweet):
    tweet = re.sub(r"\r", "", tweet)
    tweet = re.sub(r"\n", "", tweet)
    
    html = re.compile(r'<.*?>')
    tweet = html.sub(r'',tweet)
    return tweet

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

nlp = spacy.load('en_core_web_trf', disable=['parser', 'ner'])
def lemmatization(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return text

In [ ]:
df['tweet_text'] = df['tweet_text'].apply(clean_url)

df['tweet_text'] = df['tweet_text'].apply(clean)

In [ ]:
df['tweet_text'] = df['tweet_text'].apply(clean_html)

In [ ]:
df['tweet_text'] = df['tweet_text'].apply(remove_emoji)

In [ ]:
df['tweet_text'] = df['tweet_text'].apply(remove_punct)

In [13]:
df['tweet_text'] = df['tweet_text'].apply(correct_spellings)

In [ ]:
df['tweet_text'] = df['tweet_text'].apply(lemmatization)

In [ ]:
df.head()

In [ ]:
df.to_csv('out_wo_spell_lemm.csv')

In [36]:


!ls

cardiffnlp	    mallet-2.0.8.zip.2	out.csv
CMSC691_LDA.ipynb   mallet-2.0.8.zip.3	out_with_lemma.csv
Data_file	    mallet-2.0.8.zip.4	out_wo_spell_lemm.csv
data.zip	    mallet-2.0.8.zip.5	Preprocess.ipynb
figs		    mallet-2.0.8.zip.6	sentiment_analysis.ipynb
mallet-2.0.8	    mallet-2.0.8.zip.7	sentiment.csv
mallet-2.0.8.zip    mallet-2.0.8.zip.8
mallet-2.0.8.zip.1  Mallet.ipynb


In [37]:
df = pd.read_csv('out_with_lemma.csv')

In [39]:
df.head(10)

,Unnamed: 0,data,tweet_text
0,0,2020-11-23,back bed pneumonia symptom doctor i appointment tomorrow test gp office home really be gift keep give month infection
1,1,2020-11-23,grade exercise be correct advice
2,2,2020-11-23,second vitamin d clinical trial positive result covid19 longcovid coronavirus
3,3,2020-11-23,graded exercise be advise worryingly say article be bring be completely wrong advice
4,4,2020-11-23,tonight be feature story medical community be now call promise be interesting show
5,5,2020-11-23,back bed pneumonia symptom doctor i appointment tomorrow test gp office home really be gift keep give month infection
6,6,2020-11-23,thank share many itsbodypolitic patient have get creative tracking especially acute phase infection be whole other beast need systemic solution countlongcovid see discussion under here
7,7,2020-11-23,be awe inspiring More woman science month have be still ill longcovid symptom hope people take opportunity be vaccinate avoid rubbish illness immunity other mean be too risky
8,8,2020-11-23,be just be seemingly active public professional disassociation medium medical field post viral mecf feel many be deliberately ignore long stand datum draw parallel be wrong
9,9,2020-11-23,wear mask know be sick know do want make other sick kill spread other die suffer organ damage be stick


In [43]:
ddf = pd.read_csv('out_wo_spell_lemm.csv')

In [44]:
ddf.head(10)

,Unnamed: 0,data,tweet_text
0,0,2020-11-23,Back in bed with pneumonia symptoms Doctor s appointment over zoom tomorrow because they cannot test me at the GP s office or at home Covid really is the gift that keeps on giving All of this 85 months after infection LongCovid
1,1,2020-11-23,Graded exercise is not the correct advice for LongCovid
2,2,2020-11-23,Second Vitamin D clinical trial positive results via YouTube PostCovidSynd COVID19 LongCovid coronavirus
3,3,2020-11-23,Graded exercise being advised for those with LongCovid worryingly it says the article is Brought to you by the UK Government NHSEngland Telegraph this is completely wrong advice
4,4,2020-11-23,Tonight 60Minutes will be featuring a story on LongCovid LongHaulers or Post Acute Covid Syndrome as the medical community is now calling it Promises to be an interesting show Covid19 🦠
5,5,2020-11-23,Back in bed with pneumonia symptoms Doctor s appointment over zoom tomorrow because they cannot test me at the GP s office or at home Covid really is the gift that keeps on giving All of this 85 months after infection LongCovid
6,6,2020-11-23,ePatientDave ahandvanish SaraAssaf Thanks for sharing Many of our itsbodypolitic patients have gotten creative with tracking especially in the acute phase of infection Although longcovid is a whole other beast we need systemic solutions to countlongcovid See discussions under here
7,7,2020-11-23,this is awe inspiring More women in science please Plus over 8 months after having it I am still ill with LongCovid symptoms I hope people take up the opportunity to be vaccinated and avoid this rubbish illness Immunity by other means is too risky
8,8,2020-11-23,Is it just me or is there a seemingly active public and professional disassociation by the media and the medical field etc between LongCovid and post viral MECFS I feel like many not all are deliberately ignoring the long standing data and drawing parallels Am I wrong
9,9,2020-11-23,fiski70 babyowlpink lewbloch LordDrakulzen Yeshuaprovida Jackaltwinky77 th1rt3enTM MalevolenceInc1 DanaMcwhitee BrunTheBear pnschwartz kross69 thegodsareangry I wear my mask because I know I could be sick without knowing it and I do not want to make others sick with something that could kill them or that they could spread to others who might die Or suffer organ damage Or be stuck with LongCovid
